In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import mlflow
import mlflow.tensorflow
import pandas as pd
import numpy as np

In [16]:
MLFLOW_TRACKING_URI = "file:./mlruns"
TENSORBOARD_LOG_DIR = "./logs"
MODEL_SAVE_PATH = "./models/"
EPOCHS = 50
BATCH_SIZE = 64

# Initialisation
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.tensorflow.autolog()

2025/03/23 23:05:37 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.18.0, but the installed version is 2.19.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.


In [17]:
# %% Chargement des données
def load_data():
    train_data = pd.read_csv("./data/fashion-mnist_train.csv")
    test_data = pd.read_csv("./data/fashion-mnist_test.csv")

    # Préprocessing
    X_train = train_data.iloc[:,
                              1:].values.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    y_train = train_data.iloc[:, 0].values
    X_test = test_data.iloc[:,
                            1:].values.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    y_test = test_data.iloc[:, 0].values

    return (X_train, y_train), (X_test, y_test)


(X_train, y_train), (X_test, y_test) = load_data()

In [18]:
# Modèle Dense
def build_dense_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [19]:
# Modèle CNN
def build_cnn_model():
    model = models.Sequential([
        # Data augmentation
        layers.RandomRotation(0.1, input_shape=(28, 28, 1)),
        layers.RandomZoom(0.1),

        # Architecture de base
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [20]:
def get_callbacks(log_dir):
    return [
        callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(factor=0.1, patience=3),
        callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_images=True
        )
    ]

In [21]:
with mlflow.start_run(run_name="CNN_v2"):
    cnn_model = build_cnn_model()
    cnn_model.fit(
        X_train,
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        callbacks=get_callbacks(f"{TENSORBOARD_LOG_DIR}/cnn"),
        verbose=1
    )
    cnn_model.save(f"{MODEL_SAVE_PATH}/fashion_mnist_cnn_v2.h5")

d:\Tp_efrei\Deep Learning\TP deep learning\venv\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6781 - loss: 0.9415

750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 60ms/step - accuracy: 0.6782 - loss: 0.9412 - val_accuracy: 0.8217 - val_loss: 0.4925 - learning_rate: 0.0010
Epoch 2/50
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.8117 - loss: 0.5097

750/750 ━━━━━━━━━━━━━━━━━━━━ 43s 57ms/step - accuracy: 0.8118 - loss: 0.5097 - val_accuracy: 0.8466 - val_loss: 0.4331 - learning_rate: 0.0010
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.8325 - loss: 0.4571

750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 64ms/step - accuracy: 0.8325 - loss: 0.4571 - val_accuracy: 0.8535 - val_loss: 0.3978 - learning_rate: 0.0010
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8516 - loss: 0.4118

...

750/750 ━━━━━━━━━━━━━━━━━━━━ 42s 56ms/step - accuracy: 0.9178 - loss: 0.2237 - val_accuracy: 0.9261 - val_loss: 0.2081 - learning_rate: 1.0000e-05
Epoch 40/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9197 - loss: 0.2210

750/750 ━━━━━━━━━━━━━━━━━━━━ 40s 54ms/step - accuracy: 0.9197 - loss: 0.2210 - val_accuracy: 0.9257 - val_loss: 0.2080 - learning_rate: 1.0000e-05
Epoch 41/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9212 - loss: 0.2174

750/750 ━━━━━━━━━━━━━━━━━━━━ 40s 54ms/step - accuracy: 0.9212 - loss: 0.2174 - val_accuracy: 0.9256 - val_loss: 0.2079 - learning_rate: 1.0000e-05
Epoch 42/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9186 - loss: 0.2227

750/750 ━━━━━━━━━━━━━━━━━━━━ 43s 57ms/step - accuracy: 0.9186 - loss: 0.2227 - val_accuracy: 0.9258 - val_loss: 0.2076 - learning_rate: 1.0000e-05
Epoch 43/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 40s 53ms/step - accuracy: 0.9220 - loss: 0.2187 - val_accuracy: 0.9252 - val_loss: 0.2082 - learning_rate: 1.0000e-05
Epoch 44/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9202 - loss: 0.2192

750/750 ━━━━━━━━━━━━━━━━━━━━ 42s 56ms/step - accuracy: 0.9202 - loss: 0.2192 - val_accuracy: 0.9259 - val_loss: 0.2073 - learning_rate: 1.0000e-05
Epoch 45/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 40s 53ms/step - accuracy: 0.9183 - loss: 0.2211 - val_accuracy: 0.9262 - val_loss: 0.2078 - learning_rate: 1.0000e-05
Epoch 46/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 42s 56ms/step - accuracy: 0.9196 - loss: 0.2185 - val_accuracy: 0.9262 - val_loss: 0.2074 - learning_rate: 1.0000e-05
Epoch 47/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 43s 57ms/step - accuracy: 0.9188 - loss: 0.2239 - val_accuracy: 0.9262 - val_loss: 0.2074 - learning_rate: 1.0000e-05
Epoch 48/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 42s 56ms/step - accuracy: 0.9188 - loss: 0.2181 - val_accuracy: 0.9257 - val_loss: 0.2084 - learning_rate: 1.0000e-06
Epoch 49/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 42s 56ms/step - accuracy: 0.9175 - loss: 0.2285 - val_accuracy: 0.9264 - val_loss: 0.2077 - learning_rate: 1.0000e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


In [22]:
with mlflow.start_run(run_name="Dense_v2"):
    dense_model = build_dense_model()
    dense_model.fit(
        X_train,
        y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        callbacks=get_callbacks(f"{TENSORBOARD_LOG_DIR}/dense"),
        verbose=1
    )
    dense_model.save(f"{MODEL_SAVE_PATH}/fashion_mnist_dense_v2.h5")

d:\Tp_efrei\Deep Learning\TP deep learning\venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
748/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7246 - loss: 0.8096

750/750 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7248 - loss: 0.8090 - val_accuracy: 0.8397 - val_loss: 0.4467 - learning_rate: 5.0000e-04
Epoch 2/50
745/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8220 - loss: 0.4926

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8220 - loss: 0.4926 - val_accuracy: 0.8607 - val_loss: 0.3940 - learning_rate: 5.0000e-04
Epoch 3/50
744/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8356 - loss: 0.4540

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8356 - loss: 0.4540 - val_accuracy: 0.8613 - val_loss: 0.3763 - learning_rate: 5.0000e-04
Epoch 4/50
749/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8442 - loss: 0.4256

...

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9018 - loss: 0.2639 - val_accuracy: 0.8961 - val_loss: 0.2900 - learning_rate: 5.0000e-05
Epoch 33/50
743/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9029 - loss: 0.2621

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9029 - loss: 0.2621 - val_accuracy: 0.8951 - val_loss: 0.2899 - learning_rate: 5.0000e-05
Epoch 34/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9004 - loss: 0.2670 - val_accuracy: 0.8936 - val_loss: 0.2901 - learning_rate: 5.0000e-05
Epoch 35/50
747/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9008 - loss: 0.2642

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9009 - loss: 0.2641 - val_accuracy: 0.8965 - val_loss: 0.2890 - learning_rate: 5.0000e-05
Epoch 36/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9048 - loss: 0.2608 - val_accuracy: 0.8950 - val_loss: 0.2894 - learning_rate: 5.0000e-05
Epoch 37/50
746/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9014 - loss: 0.2582

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9014 - loss: 0.2582 - val_accuracy: 0.8963 - val_loss: 0.2889 - learning_rate: 5.0000e-05
Epoch 38/50
745/750 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9072 - loss: 0.2506

750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9072 - loss: 0.2506 - val_accuracy: 0.8960 - val_loss: 0.2876 - learning_rate: 5.0000e-05
Epoch 39/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9054 - loss: 0.2520 - val_accuracy: 0.8968 - val_loss: 0.2883 - learning_rate: 5.0000e-05
Epoch 40/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9075 - loss: 0.2490 - val_accuracy: 0.8966 - val_loss: 0.2898 - learning_rate: 5.0000e-05
Epoch 41/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9064 - loss: 0.2546 - val_accuracy: 0.8960 - val_loss: 0.2889 - learning_rate: 5.0000e-05
Epoch 42/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9039 - loss: 0.2512 - val_accuracy: 0.8971 - val_loss: 0.2879 - learning_rate: 5.0000e-06
Epoch 43/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9068 - loss: 0.2461 - val_accuracy: 0.8975 - val_loss: 0.2881 - learning_rate: 5.0000e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


### **Visualisation des performances avec TensorBoard et MLflow**

#### **1. Ouvrir TensorBoard**

1. **Lancer TensorBoard**  
   Dans votre terminal, exécutez la commande suivante :  
   ```bash
   tensorboard --logdir=logs
   ```

2. **Accéder à l'interface**  
   Ouvrez votre navigateur et accédez à l'adresse suivante :  
   [http://localhost:6006](http://localhost:6006)  

3. **Comparer les modèles**  
   - Dans l'onglet **Scalars**, vous pouvez visualiser les courbes de perte (`loss`) et de précision (`accuracy`) pour les ensembles d'entraînement et de validation.  
   - Comparez les performances des modèles Dense et CNN en sélectionnant les courbes correspondantes.

---

#### **2. Ouvrir MLflow**

1. **Lancer MLflow**  
   Dans votre terminal, exécutez la commande suivante :  
   ```bash
   mlflow ui
   ```

2. **Accéder à l'interface**  
   Ouvrez votre navigateur et accédez à l'adresse suivante :  
   [http://localhost:5000](http://localhost:5000)  

3. **Comparer les expériences**  
   - Dans l'interface MLflow, vous verrez les expériences enregistrées pour les modèles Dense et CNN.  
   - Cliquez sur chaque expérience pour voir les métriques (précision, perte) et les paramètres associés.  
   - Utilisez la fonctionnalité de comparaison pour analyser les performances des deux modèles.




